In [1]:
device = "mps"

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import tqdm.notebook as tqdm

In [3]:
imgs_size = 32

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(
        root='data/celeba_hq/train',
        transform=transforms.Compose([
            transforms.Resize(imgs_size),
            transforms.ToTensor(),
        ])
    ),
    batch_size=128,
    shuffle=True,
)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(
        root='data/celeba_hq/val',
        transform=transforms.Compose([
            transforms.Resize(imgs_size),
            transforms.ToTensor(),
        ])
    ),
    batch_size=128,
    shuffle=False,
)

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layers = nn.Sequential(
            # Lenet 5
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 2),
        )

    def forward(self, x):
        return self.layers(x)


model = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
def train_step(batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch
    x, y = x.to(device), y.to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test_step(batch):
    model.eval()
    x, y = batch
    x, y = x.to(device), y.to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y)
    return loss

In [6]:
# Training
pbar = tqdm.tnrange(10, unit='epoch')

for epoch in pbar:
    pbar.set_description(f'Epoch {epoch}')
    for batch in tqdm.tqdm(train_loader, leave=False, desc='Train loader', unit='batch'):
        train_loss = train_step(batch)
        pbar.set_postfix_str(f'train_loss={train_loss:.4f}, est_accuracy={torch.exp(-train_loss):.4%}')


  0%|          | 0/10 [00:00<?, ?epoch/s]

Train loader:   0%|          | 0/219 [00:00<?, ?batch/s]

Train loader:   0%|          | 0/219 [00:00<?, ?batch/s]

KeyboardInterrupt: 

In [12]:
# Testing
@torch.no_grad()
def test():
    correct = 0
    total = 0

    for batch in tqdm.tqdm(test_loader, desc='Test loader', unit='batch'):
        x, y = batch
        y_pred = model(x.to(device))
        
        _, predicted = torch.max(y_pred, 1)
        total += y.size(0)
        correct += (predicted == y.to(device)).sum().item()

    return correct / total

In [16]:
print(f"Accuracy: {test():.2%}")

Test loader:   0%|          | 0/16 [00:00<?, ?batch/s]

KeyboardInterrupt: 